In [1]:
%load_ext rich
from ipywidgets import interact
from docket_extract import *
import itertools
import os
import glob

In [2]:
ds_dir = "../../data/example_docketsheets_courtsummaries/"
fnames = list(sorted(glob.glob(os.path.join(ds_dir, "*DS_*pdf"))))
fnames

[
    '../../data/example_docketsheets_courtsummaries/DS_ MJ-17302-CR-0000035-2015.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_CP-05-CR-0000151-2006.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_CP-27-CR-0000035-2005.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_MC-51-CR-0016214-2020.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_MJ-05216-CR-0000076-2022.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_MJ-07104-CR-0000001-2023.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_MJ-07108-CR-0000086-2020.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_MJ-12301-CR-0000168-2017.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_MJ-19310-CR-0000176-2011.pdf',
    '../../data/example_docketsheets_courtsummaries/DS_MJ-26304-CR-0000183-2006.pdf'
]

In [3]:
sectioned = {fname: extract_sections(extract_text_from_pdf(fname)) for fname in fnames}

In [4]:
all_keys = set(itertools.chain(*[list(d.keys()) for d in sectioned.values()]))
key_count = {k: sum([k in d for d in sectioned.values()]) for k in all_keys}
list(sorted(key_count.items(), key=lambda x: x[1], reverse=True))

[
    ('', 10),
    ('CHARGES', 10),
    ('CALENDAR EVENTS', 10),
    ('DEFENDANT INFORMATION', 10),
    ('STATUS INFORMATION', 10),
    ('CASE PARTICIPANTS', 10),
    ('CASE INFORMATION', 10),
    ('DOCKET', 7),
    ('DISPOSITION / SENTENCING DETAILS', 7),
    ('DOCKET ENTRY INFORMATION', 7),
    ('ATTORNEY INFORMATION', 7),
    ('BAIL', 7),
    ('CONFINEMENT', 4),
    ('ENTRIES', 3),
    ('COMMONWEALTH INFORMATION            ATTORNEY INFORMATION', 3),
    ('CASE FINANCIAL INFORMATION', 3),
    ('BAIL INFORMATION', 3),
    ('DISPOSITION SENTENCING/PENALTIES', 3),
    ('CRIMINAL DOCKET', 3),
    ('RELATED CASES', 1),
    (
        'MUNICIPAL COURT OF PHILADELPHIA COUNTY                           \n                                       DOCKET',
        1
    ),
    (
        'COURT OF COMMON PLEAS OF FOREST COUNTY                            \n                                       DOCKET',
        1
    ),
    ('PAYMENT PLAN SUMMARY', 1),
    (
        'COURT OF COMMON PLEAS OF BEDFORD COUNTY                            \n                                       DOCKET',
        1
    ),
    ('WRITING', 1)
]

In [8]:
[print(sectioned[fname]["DEFENDANT INFORMATION"]) for fname in fnames]

Name:         Young, Larry Lewis       Sex:         Male                        
    Date of Birth: 08/01/1951              Race:        White                       
    Address(es):                                                                    
    Home                                                                            
    Watsontown, PA 17777                                                            
    Advised of His Right to Apply for Assignment of Counsel? No                     
    Public Defender Requested by the Defendant? No                                  
    Application Provided for Appointment of Public Defender? Yes                    
    Has the Defendant Been Fingerprinted? Yes
Date Of Birth:  10/16/1950 City/State/Zip:  Roaring Spring, PA  16673
Date Of Birth:  06/14/1959 City/State/Zip:  Oil City, PA  16301
Date Of Birth:  09/28/1988 City/State/Zip:  Philadelphia, PA  19120
Name:         Fernacz, Richard         Sex:         Male                    

[None, None, None, None, None, None, None, None, None, None]

In [9]:

def collect_defendant_information(text: str) -> dict[str, str | None]:
    """Extracts the defendant's information from the given text.
        
    Args:
        text (str): The text containing the defendant's information.
        
    Returns:
        dict: A dictionary containing the extracted information with the following keys:
        - "dob" (str or None): The date of birth of the defendant.
        - "race" (str or None): The race of the defendant.
        - "sex" (str or None): The sex of the defendant.
        - "counsel_advised" (str or None): Whether the defendant was advised of their right to apply for assignment of counsel.
        - "defender_requested" (str or None): Whether the defendant requested a public defender.
        - "application_provided" (str or None): Whether an application was provided for the appointment of a public defender.
        - "was_fingerprinted" (str or None): Whether the defendant has been fingerprinted.
    """
    # Define regular expressions to extract the required information
    dob_pattern = r"Date of Birth:\s*([\d/]+)"
    race_pattern = r"Race:\s*(\w+)"
    sex_pattern = r"Sex:\s*(\w+)"
    counsel_pattern = r"Advised of His Right to Apply for Assignment of Counsel\?\s*(\w+)"
    defender_requested_pattern = r"Public Defender Requested by the Defendant\?\s*(\w+)"
    application_provided_pattern = r"Application Provided for Appointment of Public Defender\?\s*(\w+)"
    fingerprinted_pattern = r"Has the Defendant Been Fingerprinted\?\s*(\w+)"

    # Extract the information using the regular expressions
    dob_match = re.search(dob_pattern, text, re.IGNORECASE)
    race_match = re.search(race_pattern, text, re.IGNORECASE)
    sex_match = re.search(sex_pattern, text, re.IGNORECASE)
    counsel_match = re.search(counsel_pattern, text, re.IGNORECASE)
    defender_requested_match = re.search(defender_requested_pattern, text, re.IGNORECASE)
    application_provided_match = re.search(application_provided_pattern, text, re.IGNORECASE)
    fingerprinted_match = re.search(fingerprinted_pattern, text, re.IGNORECASE)

    # Get the matched groups
    dob = dob_match.group(1) if dob_match else None
    race = race_match.group(1) if race_match else None
    sex = sex_match.group(1) if sex_match else None
    counsel = counsel_match.group(1) if counsel_match else None
    defender_requested = defender_requested_match.group(1) if defender_requested_match else None
    application_provided = application_provided_match.group(1) if application_provided_match else None
    fingerprinted = fingerprinted_match.group(1) if fingerprinted_match else None

    # Store the extracted information in a dictionary
    extracted_info = {
        "dob": dob,
        "race": race,
        "sex": sex,
        "counsel_advised": counsel,
        "defender_requested": defender_requested,
        "application_provided": application_provided,
        "was_fingerprinted": fingerprinted
    }

    # Print the extracted information
    return extracted_info

In [10]:
[collect_defendant_information(sectioned[fname]["DEFENDANT INFORMATION"]) for fname in fnames]

[
    {
        'dob': '08/01/1951',
        'race': 'White',
        'sex': 'Male',
        'counsel_advised': 'No',
        'defender_requested': 'No',
        'application_provided': 'Yes',
        'was_fingerprinted': 'Yes'
    },
    {
        'dob': '10/16/1950',
        'race': None,
        'sex': None,
        'counsel_advised': None,
        'defender_requested': None,
        'application_provided': None,
        'was_fingerprinted': None
    },
    {
        'dob': '06/14/1959',
        'race': None,
        'sex': None,
        'counsel_advised': None,
        'defender_requested': None,
        'application_provided': None,
        'was_fingerprinted': None
    },
    {
        'dob': '09/28/1988',
        'race': None,
        'sex': None,
        'counsel_advised': None,
        'defender_requested': None,
        'application_provided': None,
        'was_fingerprinted': None
    },
    {
        'dob': '10/04/1945',
        'race': 'White',
        'sex': 'Male',
        'counsel_advised': 'No',
        'defender_requested': 'No',
        'application_provided': 'No',
        'was_fingerprinted': None
    },
    {
        'dob': '05/08/1987',
        'race': 'White',
        'sex': 'Female',
        'counsel_advised': 'Yes',
        'defender_requested': 'No',
        'application_provided': 'No',
        'was_fingerprinted': 'Yes'
    },
    {
        'dob': '08/28/1986',
        'race': 'White',
        'sex': 'Male',
        'counsel_advised': 'Yes',
        'defender_requested': 'No',
        'application_provided': 'No',
        'was_fingerprinted': 'Yes'
    },
    {
        'dob': '05/21/1998',
        'race': 'White',
        'sex': 'Female',
        'counsel_advised': 'Yes',
        'defender_requested': 'No',
        'application_provided': 'No',
        'was_fingerprinted': 'Yes'
    },
    {
        'dob': '06/29/1969',
        'race': 'White',
        'sex': 'Male',
        'counsel_advised': None,
        'defender_requested': None,
        'application_provided': None,
        'was_fingerprinted': None
    },
    {
        'dob': '01/24/1964',
        'race': 'White',
        'sex': 'Male',
        'counsel_advised': 'No',
        'defender_requested': 'Yes',
        'application_provided': 'Yes',
        'was_fingerprinted': 'No'
    }
]

In [7]:
sectioned[fnames[1]]["DEFENDANT INFORMATION"]

'Date Of Birth:  10/16/1950 City/State/Zip:  Roaring Spring, PA  16673'